In [0]:
%run ./01-config

In [0]:
class Bronze():
    def __init__(self, env):
        self.Conf = Config()
        self.landing_zone = self.Conf.base_dir_data
        # self.checkpoint_base = self.Conf.base_dir_checkpoint + "/checkpoints"
        self.catalog = env
        self.db_name = self.Conf.db_name
        spark.sql(f"USE {self.catalog}.{self.db_name}")

     def load_product(self):
        schema = "product_id string, product_category_name string, product_name_length tinyint, product_description_length int, product_photos_qty int, product_weight_g tinyint, product_length_cm int, product_height_cm tinyint, product_width_cm tinyint"

        df = (spark.read
                .format("csv")
                .schema(schema)
                .option("header", "true")
                .load(self.landing_zone + "/olist_products_dataset.csv"))

        full_table_name = f"{self.catalog}.{self.db_name}.product"

        df.write \
        .format("delta") \
        .mode("overwrite") \
        .option("overwriteSchema", "true") \
        .saveAsTable(full_table_name)

        print("Batch load to product completed.")

bz = Bronze("dev")
bz.load_product()